In [28]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import math
from datetime import datetime 
import hashlib
import time
from email.message import EmailMessage
import sys
import os
path = os.getcwd()
pathing = sys.path.append(path)
sys.path.append(os.path.abspath('.')) 

# from internal_modules.email import send_email
# from internal_modules.email import sent

In [30]:
aces_individual = 'https://www.ncaa.com/stats/volleyball-men/d1/current/individual/532'

In [32]:
service = requests.get(aces_individual)

In [33]:
collecting = service.content
information_aces = BeautifulSoup(collecting, 'html.parser')
information_aces.prettify()

date_updated = '<em class="placeholder">Monday, January 27, 2025 8:08 am</em>'

In [34]:
def last_update():
    try:
        url_aces = 'https://www.ncaa.com/stats/volleyball-men/d1/current/individual/532'
        service = requests.get(url_aces)
        collecting = service.content
        information_aces = BeautifulSoup(collecting, 'html.parser')
        most_recent_date = information_aces.find_all('em')[1].text.strip()
        if len(most_recent_date) <= 1:
            return f'There has been an issue with the retrieving of the data. Contact the Author to resolve the issue.'
        else:
            time_stamp = datetime.strptime(most_recent_date, "%A, %B %d, %Y %I:%M %p")  
            return time_stamp
    except Exception as error:
        return f'There was an error at {error}, solve it to be able to parse the last updated date.'

updated = last_update()

def checking_update():
    last_known_update = updated  
    if not last_known_update:
        print(f'There has been an issue retrieving the last known update. Closing.')
        return None
    print(f'Process to check if there was an update has begun.')

    while True:
        try:
            start_time = time.time()
            
            time.sleep(604800)  
            
            print("Fetching current update timestamp...")
            current_update = updated
            
            end_time = time.time()
            elapsed_time = end_time - start_time
            
            if not current_update:
                print(f"Failed to fetch the current update time. Elapsed time: {elapsed_time:.2f} seconds. Retrying...")
            if current_update != last_known_update:
                print(f"The monitored part of the website has been updated! Elapsed time: {elapsed_time:.2f} seconds.")
                last_known_update = current_update
                break
            else:
                print(f"No update detected. Still monitoring. Elapsed time: {elapsed_time:.2f} seconds.")
        except KeyboardInterrupt:
            print("Monitoring stopped.")
        except Exception as e:
            print(f"An error occurred during monitoring: {e}")
            

            

In [86]:
def last_update():
    last_updated = information_aces.find_all('em')[0]
    most_recent_date = information_aces.find_all('em')[1].text.strip()
    most_recent_date
    time_stamp = datetime.strptime(most_recent_date, "%A, %B %d, %Y %I:%M %p") 
    print(f'The last time this website was updated was at {time_stamp}')
    return time_stamp


print(last_update())

The last time this website was updated was at 2025-02-07 09:11:00
2025-02-07 09:11:00


In [40]:
# in order to make the data frame we want to make, we will need to find all the headers and columns we were using earlier
columns_aces = information_aces.find('tr').text
columns_aces = columns_aces.replace('\n',' ').strip().split()
df = pd.DataFrame(columns = columns_aces)
df['Per Set'] = df['Per'].astype(str) + df['Set'].astype(str)
df = df.drop(['Per','Set'], axis = 1)

In [42]:
# putting everything all together

table_data = information_aces.find_all('tr')

valid_ranking = 0

for item in table_data[1:]:
    vb = item.find_all('td')
    pieced = [info.text.strip() for info in vb]
    if pieced[0] == '-':
        valid_ranking += 1
        pieced[0] = str(valid_ranking)
    else:
        valid_ranking= int(pieced[0])
    length = len(df)
    df.loc[length] = pieced


# changing the data types
df['S'] = df['S'].astype(int)
df['Aces'] = df['Aces'].astype(int)
df['Per Set'] = df['Per Set'].astype(float)
df['Rank'] = df['Rank'].astype(int)

In [62]:
df

top_five_df = df[df['Team'].isin(['BYU', 'Hawaii','Long Beach St.', 'UCLA', 'UC Irvine'])]

top_five_df

freshman = 'Fr.'

freshies = top_five_df[top_five_df['Cl'].str.contains(freshman)]

freshies

,Rank,Name,Team,Cl,Height,S,Aces,Per Set
0,1,Moni Nikolov,Long Beach St.,Fr.,6-8,27,28,1.037
10,11,Kristian Titriyski,Hawaii,Fr.,6-5,32,20,0.625
28,29,Adrien Roure,Hawaii,Fr.,6-5,35,17,0.486


In [24]:
# we usually will use .3 to .5 for the best serving teams
# will research more on the 

average = df[(df['Per Set'] > .3) & (df['Per Set'] < .5) & (df['Aces'] >= 10) & (df['S'] > int(20))]
average

df

,Rank,Name,Team,Cl,Height,S,Aces,Per Set
0,1,Moni Nikolov,Long Beach St.,Fr.,6-8,27,28,1.037
1,2,Hilir Henno,UC Irvine,Sr.,6-8,23,22,0.957
2,3,Parker Van Buren,Loyola Chicago,Sr.,6-9,34,23,0.676
3,4,Ty Carson,Menlo,Fr.,6-4,21,14,0.667
4,5,Aleksa Mandic,Sacred Heart,So.,6-5,15,10,0.667
5,6,Donovan Constable,CSUN,Sr.,6-2,32,21,0.656
6,7,Maxim Grigoriev,UC Irvine,Jr.,6-4,23,15,0.652
7,8,Bryce Dvorak,UC San Diego,Sr.,6-4,34,22,0.647
8,9,Cooper Robinson,UCLA,Sr.,6-7,25,16,0.640
9,10,Jose Gomez,Pepperdine,Fr.,6-4,16,10,0.625


In [11]:
# current status of project: 
# I am looking to do some research on the historcally best teams with their serving percentages and the overall 
# performance of the teams

# Automation should come next given the finalized of formulaic approach I decide to take

In [12]:
# next update, put key on gitignore, and include the internal module folder in git given
# it will be needed in the future of the project
